<center><span style="background-color: rgb(251, 160, 38); font-size: 32px;">  <b>NUCLIO DIGITAL SCHOOL -</b> MASTER EN DATA SCIENCE  </span></center>
    
<br>
    
<center><a href = https://nuclio.school/wp-content/uploads/2019/10/nucleoDS-newBlack.png > <img src="https://nuclio.school/wp-content/uploads/2019/10/nucleoDS-newBlack.png" width=400 height=100><a/></center>

<hr style="height:5px;border-width:0;color:orange;background-color:orange">
<center> <span style="font-size: 26px;">  Introduction to Prophet </span> </center>

<span style="font-size: 16px;">

+ Session: **Introduction to Prophet**
+ Module: **Time Series Forecasting**
+ Course: **Data Science Master 0921**
+ Professor: **Christa Santos**

</span>

<br>

<hr style="height:5px;border-width:0;color:orange;background-color:orange">

# Install and Import Prophet

Prophet is a time series data prediction algorithm / procedure based on an additive model in which non-linear trends are adjusted for annual, weekly and daily seasonality, in addition to the effects of holidays. Prophet works best with time series that have strong seasonal effects (be they weekly, monthly) and a sufficient amount of historical data where that seasonality is represented. Prophet is robust to missing data and trend changes, and it tends to handle outliers well.

This is open source software published by Facebook's Core Data Science team. It is available for download in CRAN (R) and PyPI (Python).

Prophet is an additive regression model that includes a number of highly advanced, intelligent forecasting methods, including changepoint analysis:
+ A piecewise linear or logistic growth curve trend. Prophet automatically detects changes in trends by selecting changepoints from the data.
+ A yearly seasonal component modeled using Fourier series.
+ A weekly seasonal component using dummy variables.
+ A user-provided list of important holidays.

In [ ]:
import pandas as pd
from prophet import Prophet
import plotly.express as px

# Basic Example

Let's start with a simple example from the library to understand the main procedure.

## Read the data

The input data to Prophet is always a with two columns: ds and y. The ds (datestamp) column should be in the format expected by Pandas, ideally YYYY-MM-DD for a date or YYYY-MM-DD HH: MM: SS for a timestamp. The y column must be numeric, and represents the measure that we want to forecast.

As an example, we will look at a time series of the record of daily visits to Peyton Manning's Wikipedia page. This series is a good example because it illustrates some of the characteristics of Prophet, such as multiple seasonality (weekly, monthly), changing growth rates, and the ability to model special days (such as Manning's appearances in the playoffs and the superbowl).

In [ ]:
link = 'https://raw.githubusercontent.com/facebook/prophet/master/examples/example_wp_log_peyton_manning.csv'
df = pd.read_csv(link)

In [ ]:
df.head(5)

In [ ]:
df.dtypes

In [ ]:
df['ds'] = pd.to_datetime(df['ds'])

In [ ]:
df.describe().transpose()

In [ ]:
px.line(df, x ='ds', y = 'y')

<hr style="height:5px;border-width:0;color:orange;background-color:orange">

## Build and 'fit' (train) the model

In [ ]:
m = Prophet()
m.fit(df)

## Predictions
First, we generate the dataset that we want to predict. Predictions are made in a dataframe with a column ds that contains the dates for which a prediction is to be made. A dataframe that extends into the future a specified number of days can be obtained by using the Prophet.make_future_dataframe method. By default it will also include the historical dates with which we trained the model, so we will also see the goodness of the fit of the model

In [ ]:
future = m.make_future_dataframe(periods=365)
future.tail()

Next, we generate the prediction.

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
forecast

## Visualization of results

First, we visualize the results of the model

In [ ]:
from prophet.plot import plot_plotly, plot_components_plotly

plot_plotly(m, forecast)

## Break down the results

If we want to see the components of the forecast, we can use the Prophet.plot_components method. By default, you will see the trend, the annual seasonality and the weekly seasonality of the time series. If holidays are included, they will also appear on the chart.

In [ ]:
plot_components_plotly(m, forecast)

We can also add holiday and Superbowl date information to Peyton's forecast, since we hypothesize people will visit his site more often on those dates.

In [ ]:
playoffs = pd.DataFrame({
    'holiday': 'playoff',
    'ds': pd.to_datetime(['2008-01-13', '2009-01-03', '2010-01-16',
                          '2010-01-24', '2010-02-07', '2011-01-08',
                          '2013-01-12', '2014-01-12', '2014-01-19',
                          '2014-02-02', '2015-01-11', '2016-01-17',
                          '2016-01-24', '2016-02-07']),
    'lower_window': 0,
    'upper_window': 1,
})

superbowls = pd.DataFrame({
    'holiday': 'superbowl',
    'ds': pd.to_datetime(['2010-02-07', '2014-02-02', '2016-02-07']),
    'lower_window': 0,
    'upper_window': 1,
})

holidays = pd.concat((playoffs, superbowls))

In [ ]:
m = Prophet(holidays=holidays)
forecast = m.fit(df).predict(future)

In [ ]:
forecast[(forecast['playoff'] + forecast['superbowl']).abs() > 0][
    ['ds', 'playoff', 'superbowl']][-10:]

In [ ]:
plot_components_plotly(m, forecast)

You can use a built-in collection of country-specific holidays using the add_country_holidays method. The name of the country is specified, and then major holidays for that country will be included in addition to any holidays that are specified via the holidays argument described above

In [ ]:
m = Prophet(holidays=holidays)
m.add_country_holidays(country_name='US')
m.fit(df)


You can see which holidays were included by looking at the train_holiday_names attribute of the model

In [ ]:
m.train_holiday_names

In [ ]:
forecast = m.predict(future)
fig = m.plot_components(forecast)


### Fourier Order of Seasonalities
Seasonalities are estimated using a partial Fourier sum. The number of terms in the partial sum(the order) is a parameter that determines how quickly the seasonality can change.


The default Fourier order for yearly seasonality is 10. The default values are often appropriate, but they can be increased when the seasonality needs to fit higher-frequency changes, and generally be less smooth. The Fourier order can be specified for each built-in seasonality when instantiating the model, here it is increased to 20.

In [ ]:
from prophet.plot import plot_yearly
m_yearly = Prophet(yearly_seasonality=20).fit(df)
a = plot_yearly(m_yearly)


<hr style="height:5px;border-width:0;color:orange;background-color:orange">

## Exercise to do in class

In [ ]:
import statsmodels.api as sm

data = sm.datasets.co2.load_pandas()
co2 = data.data

co2['ds'] = co2.index
co2.rename(columns={'co2': 'y'}, inplace=True)
co2.tail()

In [ ]:
# Fit the model

In [ ]:
# Forecast 15 years into the future

In [ ]:
# Populate forecast

In [ ]:
# Plot the forecast

In [ ]:
# Plot the components

Try it out on the passenger data that we saw in the last session

In [ ]:
link_pax = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv'
df_pax =  pd.read_csv(link_pax, parse_dates = ['Month'], index_col = 'Month')
df_pax.head(5)

<hr style="height:5px;border-width:0;color:orange;background-color:orange">

# More Advanced Features (Cross Validation)

In [ ]:
from prophet.diagnostics import cross_validation
df_cv = cross_validation(m, initial='730 days', period='180 days', horizon = '365 days')

The performance_metrics utility can be used to compute some useful statistics of prediction performance (yhat, yhat_lower, and yhat_upper versus y), based on the distance from the cutoff point (how far the prediction was into the future). The statistics calculated are the mean square error (MSE), the mean square error (RMSE), the mean absolute error (MAE), the mean absolute percentage error (MAPE), the mean absolute percentage error (MDAPE) and the coverage of the estimates yhat_lower and yhat_upper. They are calculated on a sliding window of the predictions in df_cv after ordering them by horizon (ds minus the cutoff). By default, 10% of the predictions will be included in each window, but this can be changed with the rolling_window argument.

In [ ]:
df_cv

In [ ]:
df_cv.cutoff.value_counts()

![cross-validation](img/diagnostics_4_0.png)

In [ ]:
cutoffs = pd.to_datetime(['2013-02-15', '2013-08-15', '2014-02-15'])
df_cv2 = cross_validation(m, cutoffs=cutoffs, horizon='365 days')


In [ ]:
from prophet.diagnostics import performance_metrics
df_p = performance_metrics(df_cv)
df_p

In [ ]:
from prophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(df_cv, metric='mape')


### Parallelizing cross validation
Cross-validation can also be run in parallel mode in Python, by setting specifying the parallel keyword. Four modes are supported

+ parallel=None (Default, no parallelization)
+ parallel="processes"
+ parallel="threads"
+ parallel="dask"

For problems that aren’t too big, we recommend using parallel="processes". It will achieve the highest performance when the parallel cross validation can be done on a single machine. For large problems, a Dask cluster can be used to do the cross validation on many machines. You will need to install Dask separately, as it will not be installed with prophet.

In [ ]:
#from dask.distributed import Client

#client = Client()  # connect to the cluster
#df_cv = cross_validation(m, initial='730 days', period='180 days', horizon='365 days', parallel="dask")

### Hyperparameter Tuning

In [ ]:
import itertools
import numpy as np
import pandas as pd

param_grid = {
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v))
              for v in itertools.product(*param_grid.values())]
rmses = []  # Store the RMSEs for each params here

# Use cross validation to evaluate all parameters
for params in all_params:
    m = Prophet(**params).fit(df)  # Fit model with given params
    df_cv = cross_validation(
        m, cutoffs=cutoffs, horizon='30 days', parallel="processes")
    df_p = performance_metrics(df_cv, rolling_window=1)
    rmses.append(df_p['rmse'].values[0])

# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
print(tuning_results)


In [ ]:
best_params = all_params[np.argmin(rmses)]
print(best_params)


### Parameters that can be tuned

+ __changepoint_prior_scale__: This is probably the most impactful parameter. It determines the flexibility of the trend, and in particular how much the trend changes at the trend changepoints. As described in this documentation, if it is too small, the trend will be underfit and variance that should have been modeled with trend changes will instead end up being handled with the noise term. If it is too large, the trend will overfit and in the most extreme case you can end up with the trend capturing yearly seasonality. The default of 0.05 works for many time series, but this could be tuned; a range of [0.001, 0.5] would likely be about right. Parameters like this (regularization penalties; this is effectively a lasso penalty) are often tuned on a log scale.

+ __seasonality_prior_scale__: This parameter controls the flexibility of the seasonality. Similarly, a large value allows the seasonality to fit large fluctuations, a small value shrinks the magnitude of the seasonality. The default is 10., which applies basically no regularization. That is because we very rarely see overfitting here (there’s inherent regularization with the fact that it is being modeled with a truncated Fourier series, so it’s essentially low-pass filtered). A reasonable range for tuning it would probably be [0.01, 10]; when set to 0.01 you should find that the magnitude of seasonality is forced to be very small. This likely also makes sense on a log scale, since it is effectively an L2 penalty like in ridge regression.

* __holidays_prior_scale__: This controls flexibility to fit holiday effects. Similar to seasonality_prior_scale, it defaults to 10.0 which applies basically no regularization, since we usually have multiple observations of holidays and can do a good job of estimating their effects. This could also be tuned on a range of [0.01, 10] as with seasonality_prior_scale.

+ __seasonality_mode__: Options are ['additive', 'multiplicative']. Default is 'additive', but many business time series will have multiplicative seasonality. This is best identified just from looking at the time series and seeing if the magnitude of seasonal fluctuations grows with the magnitude of the time series (see the documentation here on multiplicative seasonality), but when that isn’t possible, it could be tuned.

### Maybe tune?

+ __changepoint_range__: This is the proportion of the history in which the trend is allowed to change. This defaults to 0.8, 80% of the history, meaning the model will not fit any trend changes in the last 20% of the time series. This is fairly conservative, to avoid overfitting to trend changes at the very end of the time series where there isn’t enough runway left to fit it well. With a human in the loop, this is something that can be identified pretty easily visually: one can pretty clearly see if the forecast is doing a bad job in the last 20%. In a fully-automated setting, it may be beneficial to be less conservative. It likely will not be possible to tune this parameter effectively with cross validation over cutoffs as described above. The ability of the model to generalize from a trend change in the last 10% of the time series will be hard to learn from looking at earlier cutoffs that may not have trend changes in the last 10%. So, this parameter is probably better not tuned, except perhaps over a large number of time series. In that setting, [0.8, 0.95] may be a reasonable range.

### Parameters that would likely not be tuned

+ __growth__: Options are ‘linear’ and ‘logistic’. This likely will not be tuned; if there is a known saturating point and growth towards that point it will be included and the logistic trend will be used, otherwise it will be linear.

+ __changepoints__: This is for manually specifying the locations of changepoints. None by default, which automatically places them.

+ __n_changepoints__: This is the number of automatically placed changepoints. The default of 25 should be plenty to capture the trend changes in a typical time series (at least the type that Prophet would work well on anyway). Rather than increasing or decreasing the number of changepoints, it will likely be more effective to focus on increasing or decreasing the flexibility at those trend changes, which is done with changepoint_prior_scale.

+ __yearly_seasonality__: By default (‘auto’) this will turn yearly seasonality on if there is a year of data, and off otherwise. Options are [‘auto’, True, False]. If there is more than a year of data, rather than trying to turn this off during HPO, it will likely be more effective to leave it on and turn down seasonal effects by tuning seasonality_prior_scale.

+ __weekly_seasonality__: Same as for yearly_seasonality.

+ __daily_seasonality__: Same as for yearly_seasonality.

+ __holidays__: This is to pass in a dataframe of specified holidays. The holiday effects would be tuned with holidays_prior_scale.

+ __mcmc_samples__: Whether or not MCMC is used will likely be determined by factors like the length of the time series and the importance of parameter uncertainty (these considerations are described in the documentation).

+ __interval_width__: Prophet predict returns uncertainty intervals for each component, like yhat_lower and yhat_upper for the forecast yhat. These are computed as quantiles of the posterior predictive distribution, and interval_width specifies which quantiles to use. The default of 0.8 provides an 80% prediction interval. You could change that to 0.95 if you wanted a 95% interval. This will affect only the uncertainty interval, and will not change the forecast yhat at all and so does not need to be tuned.

+ __uncertainty_samples__: The uncertainty intervals are computed as quantiles from the posterior predictive interval, and the posterior predictive interval is estimated with Monte Carlo sampling. This parameter is the number of samples to use (defaults to 1000). The running time for predict will be linear in this number. Making it smaller will increase the variance (Monte Carlo error) of the uncertainty interval, and making it larger will reduce that variance. So, if the uncertainty estimates seem jagged this could be increased to further smooth them out, but it likely will not need to be changed. As with interval_width, this parameter only affects the uncertainty intervals and changing it will not affect in any way the forecast yhat; it does not need to be tuned.

+ __stan_backend__: If both pystan and cmdstanpy backends set up, the backend can be specified. The predictions will be the same, this will not be tuned.